In [1]:
import datasets
from transformers import (
    VideoMAEImageProcessor,
    VideoMAEForVideoClassification,
    VideoMAEConfig,
    VideoMAEModel
)

from transformers import (
    VideoMAEImageProcessor,
    VideoMAEForVideoClassification,
    VideoMAEPreTrainedModel,
)
from pathlib import Path
import os
os.environ['WANDB_DISABLED']='true'

In [27]:
help(VideoMAEImageProcessor)

Help on class VideoMAEImageProcessor in module transformers.models.videomae.image_processing_videomae:

class VideoMAEImageProcessor(transformers.image_processing_utils.BaseImageProcessor)
 |  VideoMAEImageProcessor(do_resize: bool = True, size: Dict[str, int] = None, resample: PIL.Image.Resampling = <Resampling.BILINEAR: 2>, do_center_crop: bool = True, crop_size: Dict[str, int] = None, do_rescale: bool = True, rescale_factor: Union[int, float] = 0.00392156862745098, do_normalize: bool = True, image_mean: Union[float, List[float], NoneType] = None, image_std: Union[float, List[float], NoneType] = None, **kwargs) -> None
 |  
 |  Constructs a VideoMAE image processor.
 |  
 |  Args:
 |      do_resize (`bool`, *optional*, defaults to `True`):
 |          Whether to resize the image's (height, width) dimensions to the specified `size`. Can be overridden by the
 |          `do_resize` parameter in the `preprocess` method.
 |      size (`Dict[str, int]` *optional*, defaults to `{"shortest_

In [ ]:
help(VideoMAEModel)

In [2]:
model_ckpt = "MCG-NJU/videomae-base"
batch_size = 8

In [3]:
target_path = "charades_HF"
datasets.config.DOWNLOADED_DATASETS_PATH = Path(target_path)
dataset = datasets.load_dataset("HuggingFaceM4/charades")

label2id = {label: i for i, label in enumerate(list(range(157)))}
id2label = {i: label for label, i in label2id.items()}

No config specified, defaulting to: charades/default
Found cached dataset charades (/home/jahid/.cache/huggingface/datasets/HuggingFaceM4___charades/default/0.0.0/c37f23dc5fc4916d250afb58fe981c5a68cfe30d9f78bc6d23a74ffa157618f7)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
config = VideoMAEConfig()

model_ckpt = "MCG-NJU/videomae-base"
image_processor = VideoMAEImageProcessor.from_pretrained(model_ckpt)
model = VideoMAEForVideoClassification.from_pretrained(
    model_ckpt,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  
)
config.problem_type = "multi_label_classification"
config.num_labels = len(label2id)
model.config = config

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from video_dataset.charades import VideoClsDataset
import random, os

In [8]:
def get_path_and_label(dataset, video_root, split='train'):
    paths = [
        os.path.join(video_root, video_info['video_id']+'.mp4')
        for video_info in dataset[split]
    ]
    labels = [video_info['labels'] for video_info in dataset[split]]

    return paths, labels

train_path, train_labels = get_path_and_label(dataset, 'charades_HF/extracted/b1114f723459b513122220bfcf30b1b7423f65be17a11b936a8d29dbf1d83eba/Charades_v1')
test_path, test_labels = get_path_and_label(dataset, 'charades_HF/extracted/b1114f723459b513122220bfcf30b1b7423f65be17a11b936a8d29dbf1d83eba/Charades_v1', split='test')

zipped_train = list(zip(train_path, train_labels))

random.seed(37)
random.shuffle(zipped_train)

split_point = int(len(zipped_train)*.9)
zipped_train, zipped_eval = zipped_train[:split_point], zipped_train[split_point: ]

train_path, train_labels = zip(*zipped_train)
eval_path, eval_labels = zip(*zipped_eval)


len(train_path), len(train_labels), len(eval_path), len(eval_labels), len(test_path), len(test_labels)

(7186, 7186, 799, 799, 1863, 1863)

In [9]:
import torch

def collate_fn(examples):
    pixel_values = torch.stack(
        [example["video"] for example in examples]
        # [example["video"].permute(1, 0, 2, 3) for example in examples]
    )
    labels = torch.stack([example["labels"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [28]:
from dataset_config import DefaultConfig

dataset_cfg = DefaultConfig()
dataset_cfg.num_sample = 1
dataset_cfg.height = 224

train_dataset = VideoClsDataset(
    data_path='',
    dataset_samples=train_path,
    label_array=train_labels,
    n_class=157,
    label2id=label2id,
    id2label=id2label,
    mode="train",
    clip_len=16,
    frame_sample_rate=2,
    crop_size=224,
    short_side_size=224,
    # new_height=224,
    # new_width=224,
    keep_aspect_ratio=True,
    num_segment=1,
    num_crop=1,
    test_num_segment=10,
    test_num_crop=3,
    args=dataset_cfg,
)

eval_dataset = VideoClsDataset(
    data_path='',
    dataset_samples=eval_path,
    label_array=eval_labels,
    n_class=157,
    label2id=label2id,
    id2label=id2label,
    mode="train",
    clip_len=16,
    frame_sample_rate=2,
    crop_size=224,
    short_side_size=224,
    # new_height=224,
    # new_width=224,
    keep_aspect_ratio=True,
    num_segment=1,
    num_crop=1,
    test_num_segment=10,
    test_num_crop=3,
    args=dataset_cfg,
)

In [29]:
for d in train_dataset:
    break

In [30]:
d['video'].shape

torch.Size([3, 16, 224, 224])

In [31]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    prefetch_factor=2,
    batch_size=batch_size
)

eval_loader = DataLoader(
    eval_dataset,
    prefetch_factor=2,
    batch_size=batch_size
)

In [ ]:
train_labels

We can also visualize the preprocessed videos for easier debugging.

In [32]:
from transformers import TrainingArguments, Trainer

model_name = model_ckpt.split("/")[-1]
new_model_name = f"{model_name}-charades"
num_epochs = 4

args = TrainingArguments(
    new_model_name,
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # push_to_hub=True,
    max_steps=(len(train_dataset)// batch_size) * num_epochs,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [33]:
import evaluate
metric = evaluate.load("accuracy")

In [34]:
# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions."""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

**A note on evaluation**:

In the [VideoMAE paper](https://arxiv.org/abs/2203.12602), the authors use the following evaluation strategy. They evaluate the model on several clips from test videos and apply different crops to those clips and report the aggregate score. However, in the interest of simplicity and brevity, we don't consider that in this tutorial.

In [ ]:
args

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [35]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    # tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [36]:
train_results = trainer.train()

  0%|          | 0/3592 [00:00<?, ?it/s]

ValueError: Make sure that the channel dimension of the pixel values match with the one set in the configuration.

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
trainer.save_model()
test_results = trainer.evaluate(test_dataset)
trainer.log_metrics("test", test_results)
trainer.save_metrics("test", test_results)
trainer.save_state()

In [ ]:
trainer.push_to_hub()

In [ ]:
trained_model = VideoMAEForVideoClassification.from_pretrained(new_model_name)

In [ ]:
sample_test_video = next(iter(test_dataset))
investigate_video(sample_test_video)

We then prepare the video as a `torch.Tensor` and run inference.

In [ ]:
def run_inference(model, video):
    """Utility to run inference given a model and test video.

    The video is assumed to be preprocessed already.
    """
    # (num_frames, num_channels, height, width)
    perumuted_sample_test_video = video.permute(1, 0, 2, 3)

    inputs = {
        "pixel_values": perumuted_sample_test_video.unsqueeze(0),
        "labels": torch.tensor(
            [sample_test_video["label"]]
        ),  # this can be skipped if you don't have labels available.
    }
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model = model.to(device)

    # forward pass
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    return logits

In [ ]:
logits = run_inference(trained_model, sample_test_video["video"])

We can now check if the model got the prediction right.

In [ ]:
display_gif(sample_test_video["video"])

In [ ]:
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

And it looks like it got it right!

You can also use this model to bring in your own videos. Check out [this Space](https://huggingface.co/spaces/sayakpaul/video-classification-ucf101-subset) to know more. The Space will also show you how to run inference for a single video file.

<br><div align=center>
    <img src="https://i.ibb.co/7nW4Rkn/sample-results.gif" width=700/>
</div>

## Next steps

Now that you've learned to train a well-performing video classification model on a custom dataset here is some homework for you:

* Increase the dataset size: include more classes and more samples per class.
* Try out different hyperparameters to study how the model converges.
* Analyze the classes for which the model fails to perform well.
* Try out a different video encoder.

Don't forget to share your models with the community =)